# Evaluating RoBERT

In [6]:
# ===========================
# 0) Install / upgrade deps
# ===========================
import sys, subprocess, math, random
from typing import Dict, Callable, Tuple, List

# def pip_install(pkgs):
#     subprocess.run([sys.executable, "-m", "pip", "install", "-U"] + pkgs, check=False)

# pip_install(["transformers", "huggingface_hub", "datasets"])

# ===========================
# 1) Imports (after upgrade)
# ===========================
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import pandas as pd
from datetime import datetime
import json
import textwrap

# ===========================
# 2) CONFIG
# ===========================
MODEL_NAME = "deepset/roberta-base-squad2"
MAX_EXAMPLES = 400
MAX_SEQ_LEN = 384
DOC_STRIDE = 128
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
random.seed(42)
print(f"Using device: {DEVICE}")

# ===========================
# 3) LOAD MODEL + TOKENIZER
# ===========================
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForQuestionAnswering.from_pretrained(MODEL_NAME).to(DEVICE)
model.eval()


Using device: cpu


RobertaForQuestionAnswering(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              

In [7]:
# ===========================
# 4) LOAD SQuAD v2 DEV, FILTER UNANSWERABLE
# ===========================
print("Loading SQuAD v2 dev split...")
ds = load_dataset("squad_v2", split="validation")

# In SQuAD 2.0, unanswerable questions have empty answers["text"]
def is_unanswerable(ex):
    return len(ex["answers"]["text"]) == 0

unans = ds.filter(is_unanswerable)
print(f"Total unanswerable in dev set: {len(unans)}")

if MAX_EXAMPLES is not None:
    unans = unans.select(range(min(MAX_EXAMPLES, len(unans))))
    print(f"Subsampled to {len(unans)} examples")


Loading SQuAD v2 dev split...
Total unanswerable in dev set: 5945
Subsampled to 400 examples


In [8]:

# ===========================
# 5) MODEL SCORING: NO-ANSWER PROB
# ===========================
def sigmoid(x: float) -> float:
    return 1.0 / (1.0 + math.exp(-x))

def get_no_answer_prob(context: str, question: str) -> Tuple[bool, float]:
    enc = tokenizer(
        question,
        context,
        truncation="only_second",
        max_length=MAX_SEQ_LEN,
        stride=DOC_STRIDE,
        return_overflowing_tokens=False,
        return_tensors="pt",
    )
    input_ids = enc["input_ids"].to(DEVICE)
    attention_mask = enc["attention_mask"].to(DEVICE)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        start_logits = outputs.start_logits[0]
        end_logits = outputs.end_logits[0]

    null_score = (start_logits[0] + end_logits[0]).item()
    seq_len = start_logits.size(0)
    best_non_null = -1e9
    for i in range(1, seq_len):
        for j in range(i, min(seq_len, i + 15)):
            s = start_logits[i].item() + end_logits[j].item()
            if s > best_non_null:
                best_non_null = s

    score_diff = null_score - best_non_null
    p_no_ans = sigmoid(score_diff)
    pred_no_ans = score_diff > 0.0
    return pred_no_ans, p_no_ans

In [9]:
# ===========================
# 6) PERTURBATIONS
# ===========================
AFRICAN_PLACE_MAP = {
    "Paris": "Lagos",
    "France": "Nigeria",
    "London": "Nairobi",
    "New York": "Accra",
    "USA": "Kenya",
    "United States": "Ghana",
    "Germany": "Ethiopia",
    "Berlin": "Kigali",
}

def natural_edit(context: str, question: str) -> Tuple[str, str]:
    sentences = [s.strip() for s in context.split('.') if s.strip()]
    if len(sentences) <= 1:
        return context, question
    drop_idx = random.randint(0, len(sentences) - 1)
    new_sentences = [s for i, s in enumerate(sentences) if i != drop_idx]
    new_context = '. '.join(new_sentences)
    if context.endswith('.'):
        new_context += '.'
    return new_context, question

def negation_attack(context: str, question: str) -> Tuple[str, str]:
    replacements = [
        (" is ", " is not "),
        (" was ", " was not "),
        (" are ", " are not "),
        (" were ", " were not "),
        (" has ", " has not "),
        (" have ", " have not "),
    ]
    new_context = context
    random.shuffle(replacements)
    for old, new in replacements:
        if old in new_context:
            new_context = new_context.replace(old, new, 1)
            break
    return new_context, question

def entity_swap(context: str, question: str) -> Tuple[str, str]:
    new_context = context
    new_question = question
    for k, v in AFRICAN_PLACE_MAP.items():
        if k in new_context:
            new_context = new_context.replace(k, v)
        if k in new_question:
            new_question = new_question.replace(k, v)
    return new_context, new_question

def paraphrase_stub(context: str, question: str) -> Tuple[str, str]:
    sentences = [s.strip() for s in context.split('.') if s.strip()]
    if len(sentences) > 1:
        random.shuffle(sentences)
    new_context = '. '.join(sentences)
    if context.endswith('.'):
        new_context += '.'
    synonym_map = {
        "big": "large",
        "large": "big",
        "small": "tiny",
        "important": "significant",
        "city": "urban area",
        "country": "nation",
    }
    for k, v in synonym_map.items():
        new_context = new_context.replace(f" {k} ", f" {v} ")
    return new_context, question

def identity(context: str, question: str) -> Tuple[str, str]:
    return context, question

PERTURBATIONS: Dict[str, Callable[[str, str], Tuple[str, str]]] = {
    "original": identity,
    "natural_edit": natural_edit,
    "negation": negation_attack,
    "entity_swap": entity_swap,
    "paraphrase": paraphrase_stub,
}


In [10]:
# ===========================
# 7) METRICS + EVALUATION
# ===========================
def evaluate_perturbation(
    dataset,
    perturb_name: str,
    perturb_fn: Callable[[str, str], Tuple[str, str]],
    max_examples: int = None,
    num_bins: int = 10,
) -> Dict[str, float]:
    n = len(dataset) if max_examples is None else min(max_examples, len(dataset))
    y_true: List[int] = []
    y_pred: List[int] = []
    p_hat: List[float] = []

    for i in range(n):
        ex = dataset[i]
        ctx = ex["context"]
        q = ex["question"]
        ctx_p, q_p = perturb_fn(ctx, q)
        pred_no_ans, p_no_ans = get_no_answer_prob(ctx_p, q_p)
        y_true.append(1)
        y_pred.append(1 if pred_no_ans else 0)
        p_hat.append(float(p_no_ans))
        if (i + 1) % 50 == 0:
            print(f"[{perturb_name}] {i+1}/{n} examples...", end="\r")

    print(f"[{perturb_name}] {n}/{n} examples.           ")
    correct = sum(yt == yp for yt, yp in zip(y_true, y_pred))
    accuracy = correct / n if n > 0 else 0.0
    hallucination_rate = 1.0 - accuracy
    avg_p_no_ans = sum(p_hat) / n if n > 0 else 0.0

    bin_bounds = [i / num_bins for i in range(num_bins + 1)]
    ece = 0.0
    for b in range(num_bins):
        lo, hi = bin_bounds[b], bin_bounds[b + 1]
        idxs = [
            idx for idx, p in enumerate(p_hat)
            if (p >= lo and (p < hi or (b == num_bins - 1 and p <= hi)))
        ]
        if not idxs:
            continue
        bin_conf = sum(p_hat[k] for k in idxs) / len(idxs)
        bin_acc = sum(y_pred[k] == y_true[k] for k in idxs) / len(idxs)
        ece += (len(idxs) / n) * abs(bin_acc - bin_conf)

    return {
        "accuracy_no_answer": accuracy,
        "hallucination_rate": hallucination_rate,
        "avg_p_no_answer": avg_p_no_ans,
        "ECE": ece,
    }


In [6]:
# ============================================
# 1) Run evaluation for all perturbations
# ============================================
results = {}

for name, fn in PERTURBATIONS.items():
    print(f"\n=== Evaluating perturbation: {name} ===")
    metrics = evaluate_perturbation(unans, name, fn, max_examples=MAX_EXAMPLES)
    results[name] = metrics
    print(f"Results for {name}:")
    for k, v in metrics.items():
        print(f"  {k:20s}: {v:.4f}")


=== Evaluating perturbation: original ===
[original] 400/400 examples.           
Results for original:
  accuracy_no_answer  : 0.8300
  hallucination_rate  : 0.1700
  avg_p_no_answer     : 0.8339
  ECE                 : 0.0323

=== Evaluating perturbation: natural_edit ===
[natural_edit] 400/400 examples.           
Results for natural_edit:
  accuracy_no_answer  : 0.8875
  hallucination_rate  : 0.1125
  avg_p_no_answer     : 0.8791
  ECE                 : 0.0295

=== Evaluating perturbation: negation ===
[negation] 400/400 examples.           
Results for negation:
  accuracy_no_answer  : 0.8375
  hallucination_rate  : 0.1625
  avg_p_no_answer     : 0.8324
  ECE                 : 0.0336

=== Evaluating perturbation: entity_swap ===
[entity_swap] 400/400 examples.           
Results for entity_swap:
  accuracy_no_answer  : 0.8375
  hallucination_rate  : 0.1625
  avg_p_no_answer     : 0.8366
  ECE                 : 0.0313

=== Evaluating perturbation: paraphrase ===
[paraphrase] 400/4

In [7]:
# ============================================
# 2) Convert to DataFrame for pretty display
# ============================================
df = pd.DataFrame.from_dict(results, orient="index")
df = df[["accuracy_no_answer", "hallucination_rate", "avg_p_no_answer", "ECE"]]  # column order
df = df.sort_index()
df_rounded = df.round(4)

In [8]:
print("\n\n================= SUMMARY TABLE =================")
print(df_rounded.to_markdown(tablefmt="github"))
print("\nDone.")

# ============================================
# 3) Save results to disk (CSV + JSON)
# ============================================
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

csv_path = f"mrc_robustness_results_{timestamp}.csv"
json_path = f"mrc_robustness_results_{timestamp}.json"

df.to_csv(csv_path, index=True)
with open(json_path, "w") as f:
    json.dump(results, f, indent=2)

print(f"\nSaved results to:\n  - {csv_path}\n  - {json_path}")

# ============================================
# 4) Save a human-readable experiment summary
# ============================================
experiment_config = {
    "model_name": MODEL_NAME,
    "dataset": "SQuAD v2 validation (unanswerable subset)",
    "max_examples": MAX_EXAMPLES,
    "num_unanswerable_used": len(unans),
    "perturbations": list(PERTURBATIONS.keys()),
}

summary_lines = []

summary_lines.append("# MRC Robustness Experiment Summary\n")
summary_lines.append(f"**Timestamp:** {timestamp}")
summary_lines.append(f"**Model:** `{experiment_config['model_name']}`")
summary_lines.append(f"**Base dataset:** {experiment_config['dataset']}")
summary_lines.append(f"**Number of unanswerable examples used:** {experiment_config['num_unanswerable_used']}")
summary_lines.append(f"**Max examples cap:** {experiment_config['max_examples']}")
summary_lines.append("**Perturbations applied:**")
for p in experiment_config["perturbations"]:
    summary_lines.append(f"- {p}")

summary_lines.append("\n## Metrics per perturbation\n")
summary_lines.append(df_rounded.to_markdown(tablefmt="github"))

summary_lines.append("\n## Methodology (short description)\n")
methodology = """
We evaluate the robustness of a modern reading comprehension (MRC) model on unanswerable
questions from the SQuAD v2 validation set. We first filter the dataset to keep only
examples where the ground-truth answer is empty (i.e., the question is unanswerable
given the context).

For each context–question pair, we generate multiple perturbed versions of the context
using the following transformations:
1. original: no perturbation, original SQuAD v2 context.
2. natural_edit: delete one sentence from the context to simulate a prior revision.
3. negation: insert simple negations (e.g., 'is' -> 'is not') into the context.
4. entity_swap: replace some common locations with African-origin entities.
5. paraphrase: lightly shuffle sentences and swap simple synonyms as a stub for paraphrasing.

For each (context, question) pair under each perturbation, we run the QA model and
compute the null-vs-span score difference:
    score_diff = (start_logits[CLS] + end_logits[CLS]) - best_non_null_span_score.
We interpret:
    - score_diff > 0  => model predicts "no answer".
    - score_diff <= 0 => model predicts some answer span (hallucination in this setup).

Since all questions are truly unanswerable, the ideal model always predicts "no answer".
We report:
    - accuracy_no_answer: fraction of examples where the model predicts "no answer".
    - hallucination_rate: fraction where the model predicts a span (1 - accuracy).
    - avg_p_no_answer: average sigmoid(score_diff), interpreted as the model's
      confidence in "no answer".
    - ECE: a simple expected calibration error over p(no-answer), using 10 bins.
"""
summary_lines.append(textwrap.dedent(methodology).strip() + "\n")

summary_path = f"mrc_robustness_experiment_summary_{timestamp}.txt"
with open(summary_path, "w") as f:
    f.write("\n".join(summary_lines))

print(f"\nSaved experiment summary to:\n  - {summary_path}")




================= SUMMARY TABLE =================
|              |   accuracy_no_answer |   hallucination_rate |   avg_p_no_answer |    ECE |
|--------------|----------------------|----------------------|-------------------|--------|
| entity_swap  |               0.8375 |               0.1625 |            0.8366 | 0.0313 |
| natural_edit |               0.8875 |               0.1125 |            0.8791 | 0.0295 |
| negation     |               0.8375 |               0.1625 |            0.8324 | 0.0336 |
| original     |               0.83   |               0.17   |            0.8339 | 0.0323 |
| paraphrase   |               0.8425 |               0.1575 |            0.8412 | 0.0289 |

Done.

Saved results to:
  - mrc_robustness_results_20251126_011421.csv
  - mrc_robustness_results_20251126_011421.json

Saved experiment summary to:
  - mrc_robustness_experiment_summary_20251126_011421.txt


# Fine-Tuning BERT

In [11]:
import sys, subprocess, os, math, random, json, textwrap
from typing import Dict, Callable, Tuple, List
from datetime import datetime

def pip_install(pkgs):
    subprocess.run([sys.executable, "-m", "pip", "install", "-U"] + pkgs, check=False)

# Install deps (no-op if already present)
pip_install(["openai", "python-dotenv", "datasets", "pandas"])

from dotenv import load_dotenv
import pandas as pd
from openai import OpenAI

In [ ]:
# --------------------------
# Load API key from .env
# --------------------------
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise RuntimeError("OPENAI_API_KEY not found. Make sure .env has OPENAI_API_KEY=sk-proj-...")

client = OpenAI(api_key=api_key)

# Choose your GPT model here
# If you have access to GPT-5, use e.g. "gpt-5" or "gpt-5-mini".
# Otherwise, "gpt-4.1-mini" is a solid default.
GPT_MODEL = "gpt-5"

print(f"Using GPT model: {GPT_MODEL}")

# ============================================================
# 1) Dataset + Perturbations (re-use if already defined)
# ============================================================

from datasets import load_dataset

MAX_GPT_EXAMPLES = 200  # lower than 400 to save cost; bump if you want

# Try to reuse `unans` if defined; otherwise load from scratch
unans  # type: ignore[name-defined]
print(f"Reusing existing unans dataset with {len(unans)} examples.")


# Try to reuse PERTURBATIONS if defined; otherwise recreate simple ones
PERTURBATIONS  # type: ignore[name-defined]
print("Reusing existing PERTURBATIONS dict.")

random.seed(42)

# ============================================================
# 2) GPT-based "no-answer" prediction
# ============================================================

def gpt_predict_no_answer(context: str, question: str) -> Tuple[bool, float]:
    """
    Ask GPT to perform RC with an explicit "NO_ANSWER" option.

    Protocol:
      - If the answer is present in the context, GPT should reply with a short answer span.
      - If not, GPT must reply with **exactly** 'NO_ANSWER'.

    We then:
      - pred_no_ans = True if cleaned output == NO_ANSWER
      - p_no_ans = 1.0 if pred_no_ans else 0.0 (binary confidence proxy)
    """
    system_msg = {
        "role": "system",
        "content": (
            "You are a strict reading comprehension model. "
            "You MUST follow these rules exactly:\n"
            "1. You are given a context and a question.\n"
            "2. If the question CAN be answered using ONLY the context, "
            "   reply with a short answer span copied exactly from the context.\n"
            "3. If the question CANNOT be answered from the context, reply with "
            "   EXACTLY the token: NO_ANSWER (all caps, no punctuation, nothing else).\n"
        ),
    }

    user_msg = {
        "role": "user",
        "content": (
            f"Context:\n{context}\n\n"
            f"Question:\n{question}\n\n"
            "Remember: if the answer is not present in the context, reply with NO_ANSWER."
        ),
    }

    resp = client.chat.completions.create(
        model=GPT_MODEL,
        messages=[system_msg, user_msg],
        temperature=0.0,
        max_completion_tokens=32,
    )

    raw = resp.choices[0].message.content.strip()
    cleaned = raw.strip().lower().replace(".", "").replace(" ", "_")

    # We accept a few variants just in case: "NO_ANSWER", "NO ANSWER"
    pred_no_ans = cleaned in {"no_answer"}

    p_no_ans = 1.0 if pred_no_ans else 0.0  # binary confidence proxy
    return pred_no_ans, p_no_ans

# ============================================================
# 3) Evaluation (same metrics as RoBERTa version)
# ============================================================

def evaluate_perturbation_gpt(
    dataset,
    perturb_name: str,
    perturb_fn: Callable[[str, str], Tuple[str, str]],
    max_examples: int = None,
    num_bins: int = 10,
) -> Dict[str, float]:
    n = len(dataset) if max_examples is None else min(max_examples, len(dataset))
    y_true: List[int] = []
    y_pred: List[int] = []
    p_hat: List[float] = []

    for i in range(n):
        ex = dataset[i]
        ctx = ex["context"]
        q = ex["question"]

        ctx_p, q_p = perturb_fn(ctx, q)
        pred_no_ans, p_no_ans = gpt_predict_no_answer(ctx_p, q_p)

        y_true.append(1)  # all unanswerable
        y_pred.append(1 if pred_no_ans else 0)
        p_hat.append(float(p_no_ans))

        if (i + 1) % 20 == 0:
            print(f"[{perturb_name}] Processed {i+1}/{n} examples...", end="\r")

    print(f"[{perturb_name}] Processed {n}/{n} examples.           ")

    correct = sum(yt == yp for yt, yp in zip(y_true, y_pred))
    accuracy = correct / n if n > 0 else 0.0
    hallucination_rate = 1.0 - accuracy
    avg_p_no_ans = sum(p_hat) / n if n > 0 else 0.0

    # Simple ECE over p_hat (which is 0/1 here, so still meaningful)
    bin_bounds = [i / num_bins for i in range(num_bins + 1)]
    ece = 0.0
    for b in range(num_bins):
        lo, hi = bin_bounds[b], bin_bounds[b + 1]
        idxs = [
            idx for idx, p in enumerate(p_hat)
            if (p >= lo and (p < hi or (b == num_bins - 1 and p <= hi)))
        ]
        if not idxs:
            continue
        bin_conf = sum(p_hat[k] for k in idxs) / len(idxs)
        bin_acc = sum(y_pred[k] == y_true[k] for k in idxs) / len(idxs)
        ece += (len(idxs) / n) * abs(bin_acc - bin_conf)

    return {
        "accuracy_no_answer": accuracy,
        "hallucination_rate": hallucination_rate,
        "avg_p_no_answer": avg_p_no_ans,
        "ECE": ece,
    }

# ============================================================
# 4) Run evaluation for all perturbations
# ============================================================

gpt_results = {}
for name, fn in PERTURBATIONS.items():
    print(f"\n=== [GPT] Evaluating perturbation: {name} ===")
    metrics = evaluate_perturbation_gpt(unans, name, fn, max_examples=MAX_GPT_EXAMPLES)
    gpt_results[name] = metrics
    print(f"Results for {name}:")
    for k, v in metrics.items():
        print(f"  {k:20s}: {v:.4f}")

# ============================================================
# 5) Pretty summary + save artifacts
# ============================================================

df_gpt = pd.DataFrame.from_dict(gpt_results, orient="index")
df_gpt = df_gpt[["accuracy_no_answer", "hallucination_rate", "avg_p_no_answer", "ECE"]]
df_gpt = df_gpt.sort_index()
df_gpt_rounded = df_gpt.round(4)

print("\n\n================= GPT SUMMARY TABLE =================")
print(df_gpt_rounded.to_markdown(tablefmt="github"))

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

csv_path = f"gpt_mrc_robustness_results_{timestamp}.csv"
json_path = f"gpt_mrc_robustness_results_{timestamp}.json"

df_gpt.to_csv(csv_path, index=True)
with open(json_path, "w") as f:
    json.dump(gpt_results, f, indent=2)

print(f"\nSaved GPT results to:\n  - {csv_path}\n  - {json_path}")

# ------------------ Summary text file ------------------
summary_lines = []
summary_lines.append("# GPT MRC Robustness Experiment Summary\n")
summary_lines.append(f"**Timestamp:** {timestamp}")
summary_lines.append(f"**GPT Model:** `{GPT_MODEL}`")
summary_lines.append("**Base dataset:** SQuAD v2 validation (unanswerable subset)")
summary_lines.append(f"**Number of unanswerable examples used:** {len(unans)}")
summary_lines.append(f"**Max examples cap (GPT):** {MAX_GPT_EXAMPLES}")
summary_lines.append("**Perturbations applied:**")
for p in PERTURBATIONS.keys():
    summary_lines.append(f"- {p}")

summary_lines.append("\n## Metrics per perturbation\n")
summary_lines.append(df_gpt_rounded.to_markdown(tablefmt="github"))

methodology = f"""
We evaluate the robustness of a large language model (`{GPT_MODEL}`) on
unanswerable questions from the SQuAD v2 validation set. As in the RoBERTa
baseline experiment, we first filter the dataset to keep only examples where
the ground-truth answer is empty, meaning the question is unanswerable given
the context.

For each context–question pair, we generate multiple perturbed versions of the
context using the same transformations as before:
1. original: no perturbation, original SQuAD v2 context.
2. natural_edit: delete one sentence from the context to simulate a prior revision.
3. negation: insert simple negations (e.g., 'is' -> 'is not') into the context.
4. entity_swap: replace common locations with African-origin entities.
5. paraphrase: lightly shuffle sentences and swap simple synonyms as a stub for paraphrasing.

Instead of span extraction logits, we use GPT as a generative QA system with
an explicit protocol. The model receives the context and question, and is
instructed to:
  - reply with a short answer span *copied from the context* if the question
    is answerable, and
  - reply with the exact token 'NO_ANSWER' if the question is not answerable
    from the context.

Since in this experiment all questions are truly unanswerable, the ideal model
should always output 'NO_ANSWER'. For each example, we post-process the model
output:
  - If the cleaned output equals 'NO_ANSWER', we treat this as a correct
    'no-answer' prediction.
  - Otherwise, we treat it as a hallucinated answer span.

We report:
  - accuracy_no_answer: fraction of examples where GPT outputs 'NO_ANSWER'.
  - hallucination_rate: fraction where GPT outputs some other text
    (1 - accuracy).
  - avg_p_no_answer: average predicted probability of 'no-answer'; here we
    approximate this by 1.0 for 'NO_ANSWER' predictions and 0.0 otherwise.
  - ECE: a simple expected calibration error over p(no-answer), using 10 bins.

These metrics are directly comparable in spirit to the RoBERTa baseline and
allow us to analyze how a modern GPT model behaves on unanswerable queries and
under the same perturbations.
"""
summary_lines.append("\n## Methodology (short description)\n")
summary_lines.append(textwrap.dedent(methodology).strip() + "\n")

summary_path = f"gpt_mrc_robustness_experiment_summary_{timestamp}.txt"
with open(summary_path, "w") as f:
    f.write("\n".join(summary_lines))

print(f"\nSaved GPT experiment summary to:\n  - {summary_path}\n")
print("Done.")


Using GPT model: gpt-5
Reusing existing unans dataset with 400 examples.
Reusing existing PERTURBATIONS dict.

=== [GPT] Evaluating perturbation: original ===


BadRequestError: Error code: 400 - {'error': {'message': "Unsupported parameter: 'max_tokens' is not supported with this model. Use 'max_completion_tokens' instead.", 'type': 'invalid_request_error', 'param': 'max_tokens', 'code': 'unsupported_parameter'}}